### Importing Modules

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing import image
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import google.generativeai as palm

C:\Users\kommi\anaconda3\envs\Tf-new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import datasets

In [3]:
df=pd.read_csv('vitamins.csv')

In [4]:
df.head()

,Name,Calories (kcal),Carbohydrates (g),Fiber (g),Vitamin C (mg),Vitamin A (IU),Vitamin B6 (mg),Potassium (mg)
0,banana,105,27.0,3.1,10.3,76,0.4,422
1,apple,95,25.0,4.4,8.4,98,0.1,195
2,pear,101,27.0,5.5,4.3,80,0.1,116
3,grapes,62,16.0,0.8,3.2,66,0.1,191
4,orange,62,15.0,3.1,69.7,225,0.1,237


### Configure API key

#### check Readme File on how to obtain a Palm api Key

In [5]:
palm.configure(api_key="YOUR_PALM_API_KEY")

### Loading our Pre-trained Model

In [6]:
model = tf.keras.models.load_model('finetuned_inceptionv3_model.h5')

### Importing Labels

#### we only need all the class labels no need of whole dataset , to avoid loading the whole dataset , we will make class labels text file for the future use

In [7]:
# Load the class labels from the file
with open('class_labels.txt', 'r') as file:
    class_labels = [line.strip() for line in file.readlines()]

### Functions

#### Predict Image Function will pre process the uploaded image and predict the class of image

In [8]:
def predict_image(image_path):
    img_path = image_path
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    # Make a prediction
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_labels[predicted_class_index] # assuming you have the train_generator
    return predicted_class

#### Nutrition facts Function Produce Nutrition and Fun facts 

Nutrition facts Function uses predicted varaiable and searches its nutrition info from csv and also using text-model from Palm api it retrieves Fun facts about classified Label 

In [12]:
def nutrition_facts(name):
    if name in df['Name'].values:
        row = df.loc[df['Name'] == name]
        for col in df.columns:
            print(f"{col}:")
            print(f"{row[col].values[0]}")
            print()
        prompt = f"2 fun facts about {name}"
        model='models/text-bison-001'
        print(f"Few facts about {name}:")
        completion= palm.generate_text(
        model=model,
        prompt=prompt,
        temperature=0.3,
        max_output_tokens=80, )
        print(completion.result)
    else:
        print(f"The value '{name}' does not exist in the dataset.")

Main Function is to call both predict_image ,nutrition_facts

In [13]:
def main(x):
    y=predict_image(x)
    print("The Predicted Fruit/Veggie :",y)
    print("Vitamin Details:")
    print(nutrition_facts(y))

In [14]:
x=input("Enter the File path: ")
main(x)

Enter the File path: C:\\Users\\kommi\\Downloads\\tom.jpg
1/1 [==============================] - 0s 199ms/step
The Predicted Fruit/Veggie : tomato
Vitamin Details:
Name:
tomato

Calories (kcal):
18

Carbohydrates (g):
4.0

Fiber (g):
1.2

Vitamin C (mg):
14.0

Vitamin A (IU):
833

Vitamin B6 (mg):
0.1

Potassium (mg):
237

Few facts about tomato:
1. The tomato is a fruit, not a vegetable.
2. Tomatoes are one of the most popular fruits in the world.
None
